In [6]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#data collection
dataset=pd.read_csv("Social_Network_Ads.csv")
print(dataset)

#preprocess
#drop/remove user ID, model creation
dataset=dataset.drop("User ID",axis=1)
dataset

#preprocess Gender to 0 or 1
dataset=pd.get_dummies(dataset,drop_first=True).astype(int)
print(dataset.columns)

#Balanced Dataset or Imbalanced Dataset
print(dataset["Purchased"].value_counts())

#input output split
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent=dataset[['Purchased']]

print(independent.shape)
print(dependent.shape)

#Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

#preprocess standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

      User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]
Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')
Purchased
0    257
1    143
Name: count, dtype: int64
(400, 3)
(400, 1)


In [7]:
#Model creation
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
from sklearn.tree import DecisionTreeClassifier

#Model creation using Grid
from sklearn.model_selection import GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],  
    'max_features': [None, 'sqrt', 'log2'],        
    'splitter': ['best', 'random']
}
classifier = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
# fitting the model for grid search 
classifier.fit(X_train, y_train) 

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [8]:
#Evaluation Matrix: Confusion_matrix
re=classifier.cv_results_
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Evaluation Matrix: calculate P R F
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

[[71  8]
 [ 3 38]]
              precision    recall  f1-score   support

           0       0.96      0.90      0.93        79
           1       0.83      0.93      0.87        41

    accuracy                           0.91       120
   macro avg       0.89      0.91      0.90       120
weighted avg       0.91      0.91      0.91       120



In [9]:
# print best parameter after tuning 
#print(classifier.best_params_) 
re=classifier.cv_results_
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(classifier.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}: 0.9094696115994291


In [10]:
#Roc = ROC (Receiver Operating Characteristic) Curve
#AUC (Area Under the Curve)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,classifier.predict_proba(X_test)[:,1])

np.float64(0.9127817227539364)

In [11]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005150,0.000572,0.011938,0.001171,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.799537,0.821429,0.804584,0.876368,0.909115,0.842206,0.043155,9
1,0.005947,0.001119,0.012152,0.000949,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.835985,0.804584,0.804584,0.874356,0.911105,0.846123,0.041409,8
2,0.003139,0.001063,0.009830,0.002431,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.850032,0.804584,0.821429,0.859025,0.870721,0.841158,0.024490,10
3,0.002588,0.000445,0.014921,0.003449,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.872432,0.838326,0.752381,0.891667,0.780228,0.827007,0.053160,12
4,0.004894,0.001430,0.014909,0.001984,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.819142,0.802399,0.789152,0.805721,0.833784,0.810040,0.015231,16
5,0.003771,0.001059,0.012766,0.001955,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.819142,0.785714,0.859435,0.787433,0.875548,0.825454,0.036684,13
6,0.005065,0.001098,0.010286,0.000698,entropy,None,best,"{'criterion': 'entropy', 'max_features': None,...",0.819142,0.840114,0.807155,0.892857,0.927778,0.857409,0.045820,1
7,0.003919,0.001087,0.011486,0.001260,entropy,None,random,"{'criterion': 'entropy', 'max_features': None,...",0.855314,0.819142,0.782971,0.911105,0.874356,0.848578,0.044250,7
8,0.003346,0.000738,0.011741,0.000894,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.835985,0.893878,0.821429,0.839990,0.874356,0.853127,0.026768,4
9,0.003625,0.000873,0.010947,0.001258,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.840114,0.802399,0.807155,0.776316,0.890114,0.823219,0.039120,14


In [12]:
#save model
import pickle
filename="final_Sav_Model.sav"
pickle.dump(classifier,open(filename,'wb'))
load_model=pickle.load(open("final_Sav_Model.sav",'rb'))

#apply StandardScaler to user input
preinput=sc.transform([[46,41000,1]])
print(preinput)
result=load_model.predict(preinput)
result

[[ 0.79762394 -0.8330751   0.99288247]]


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])